# 階層分析法「AHP」の考え方とPythonによる実装
## https://logics-of-blue.com/ahp-concept-and-implementation-by-python

In [1]:
# ライブラリを読み込む
from sympy import * 

# 出力を美しくする
init_printing()

In [2]:
# sympyのバージョン
import sympy
print(sympy.__version__)

1.3


## 味と価格の2つの基準の重みを計算する

In [3]:
# 基準の行列
eval_mat_1 = Matrix(    
    [[1   , 3,],
     [1/3 , 1]]
)

In [4]:
# 固有値と固有ベクトル
eigen_val_vects_1 = eval_mat_1.eigenvects()
eigen_val_vects_1

⎡⎛      ⎡⎡-3.0⎤⎤⎞  ⎛        ⎡⎡3.0⎤⎤⎞⎤
⎢⎜0, 1, ⎢⎢    ⎥⎥⎟, ⎜2.0, 1, ⎢⎢   ⎥⎥⎟⎥
⎣⎝      ⎣⎣1.0 ⎦⎦⎠  ⎝        ⎣⎣1.0⎦⎦⎠⎦

In [5]:
# 固有値と多重度、固有ベクトルのインデックスを保存しておく
EIGEN_VAL_IDX = 0
MULUTIPLICITY_IDX = 1
EIGEN_VEC_IDX = 2

In [6]:
# 最大固有値とその時の多重度・固有ベクトルを取得
max(eigen_val_vects_1, key=(lambda x: x[EIGEN_VAL_IDX]))

⎛        ⎡⎡3.0⎤⎤⎞
⎜2.0, 1, ⎢⎢   ⎥⎥⎟
⎝        ⎣⎣1.0⎦⎦⎠

In [7]:
# 要素ごとに分けて取得
eigen_val, multiplicity, eigen_vec = max(eigen_val_vects_1, key=(lambda x: x[EIGEN_VAL_IDX]))
eigen_vec

⎡⎡3.0⎤⎤
⎢⎢   ⎥⎥
⎣⎣1.0⎦⎦

In [8]:
# 固有ベクトルは、なぜかlistになっている
type(eigen_vec)

list

In [9]:
# 最初の要素を取得
eigen_vec[0]

⎡3.0⎤
⎢   ⎥
⎣1.0⎦

In [10]:
# こちらはちゃんと行列になっている
type(eigen_vec[0])

sympy.matrices.dense.MutableDenseMatrix

In [11]:
# 重みの計算
eigen_vec[0] / sum(eigen_vec[0])

⎡0.75⎤
⎢    ⎥
⎣0.25⎦

### 比較行列から重みを計算するための関数を作る

In [12]:
# 比較行列から重みベクトルを計算する関数
def calc_ahp_weight_vec(comparison_mat):
    '''
    比較行列から重みベクトルを計算する関数

    Arguments:

    comparison_mat -- 比較行列
    
    '''
    
    # 固有値のインデックス
    EIGEN_VAL_IDX = 0
    
    # sympyの関数を使って、固有値と固有ベクトルを得る
    eigen_val_vects = comparison_mat.eigenvects()
    
    # 最大固有値とその時の多重度・固有ベクトルを取得
    eigen_val, multiplicity, eigen_vec = max(eigen_val_vects, key=(lambda x: x[EIGEN_VAL_IDX]))
    
    # 重みの合計が1になるように標準化
    weight_vec = eigen_vec[0] / sum(eigen_vec[0])
    
    # 重みを返す
    return(weight_vec)

In [13]:
# 確認
calc_ahp_weight_vec(eval_mat_1)

⎡0.75⎤
⎢    ⎥
⎣0.25⎦

### 寿司と牛丼問題を解決する

In [14]:
# 基準の行列
# 要素1が味
# 要素2が価格
eval_mat_1 = Matrix(    
    [[1   , 3,],
     [1/3 , 1]]
)

# 味に関する、お寿司と牛丼の比較行列
# 要素1がお寿司
# 要素2が牛丼
taste_mat_1 = Matrix(    
    [[1   , 5,],
     [1/5 , 1]]
)

# 価格に関する、お寿司と牛丼の比較行列
# 要素1がお寿司
# 要素2が牛丼
price_mat_1 = Matrix(    
    [[1 , 1/9,],
     [9 ,   1]]
)


In [15]:
# 基準の重みベクトル
eval_weight_1 = calc_ahp_weight_vec(eval_mat_1)
eval_weight_1

⎡0.75⎤
⎢    ⎥
⎣0.25⎦

In [16]:
# 味に関する、お寿司と牛丼の重みベクトル
taste_weight_1 = calc_ahp_weight_vec(taste_mat_1)
N(taste_weight_1, 2)

⎡0.83⎤
⎢    ⎥
⎣0.17⎦

In [17]:
# 価格に関する、お寿司と牛丼の重みベクトル
price_weight_1 = calc_ahp_weight_vec(price_mat_1)
price_weight_1

⎡0.1⎤
⎢   ⎥
⎣0.9⎦

In [18]:
# お寿司と牛丼の結果を結合
join_mat_1 = taste_weight_1.row_join(price_weight_1)
N(join_mat_1, 2)

⎡0.83  0.1⎤
⎢         ⎥
⎣0.17  0.9⎦

In [19]:
# 総合評価
# 要素1がお寿司
# 要素2が牛丼
join_mat_1 * eval_weight_1

⎡0.65⎤
⎢    ⎥
⎣0.35⎦

# 味・価格・待ち時間の3つの基準があるときのAHP

### 推移性が満たされている例

In [20]:
# 基準の行列
# 要素1が味
# 要素2が価格
# 要素3が待ち時間
eval_mat_2 = Matrix(    
    [[1   , 3  , 15],
     [1/3 , 1  , 5],
     [1/15, 1/5, 1]]
)

In [21]:
# 最大固有値とその時の多重度・固有ベクトルを取得
eigen_val_vects_2 = eval_mat_2.eigenvects()
max(eigen_val_vects_2, key=(lambda x: x[EIGEN_VAL_IDX]))

⎛        ⎡⎡15.0⎤⎤⎞
⎜        ⎢⎢    ⎥⎥⎟
⎜3.0, 1, ⎢⎢5.0 ⎥⎥⎟
⎜        ⎢⎢    ⎥⎥⎟
⎝        ⎣⎣1.0 ⎦⎦⎠

In [22]:
# 基準の重みベクトル
eval_weight_2 = calc_ahp_weight_vec(eval_mat_2)
N(eval_weight_2, 2)

⎡0.71 ⎤
⎢     ⎥
⎢0.24 ⎥
⎢     ⎥
⎣0.048⎦

### 推移性が満たされていない例

In [23]:
# 基準の行列
# 要素1が味
# 要素2が価格
# 要素3が待ち時間
eval_mat_3 = Matrix(    
    [[1   , 3  , 1],
     [1/3 , 1  , 5],
     [1   , 1/5, 1]]
)

In [24]:
eigen_val_vects_3 = eval_mat_3.eigenvects()
eigen_val_vects_3

⎡⎛                                            ⎡⎡-1.23310603716524 - 2.13580230
⎢⎜                                            ⎢⎢                              
⎢⎜-0.435846103684348 - 1.78464621154911⋅ⅈ, 1, ⎢⎢-1.01370033259557 + 1.75578047
⎢⎜                                            ⎢⎢                              
⎣⎝                                            ⎣⎣                 1.0          

74901⋅ⅈ ⎤⎤⎞  ⎛                                            ⎡⎡-1.23310603716524 
        ⎥⎥⎟  ⎜                                            ⎢⎢                  
970499⋅ⅈ⎥⎥⎟, ⎜-0.435846103684348 + 1.78464621154911⋅ⅈ, 1, ⎢⎢-1.01370033259557 
        ⎥⎥⎟  ⎜                                            ⎢⎢                  
        ⎦⎦⎠  ⎝                                            ⎣⎣                 1

+ 2.1358023074901⋅ⅈ ⎤⎤⎞  ⎛                    ⎡⎡2.46621207433047⎤⎤⎞⎤
                    ⎥⎥⎟  ⎜                    ⎢⎢                ⎥⎥⎟⎥
- 1.75578047970499⋅ⅈ⎥⎥⎟, ⎜3.8716922073687, 1, ⎢⎢2.02740066519113⎥⎥⎟⎥
 

In [25]:
# 最大固有値とその時の多重度・固有ベクトルを取得
# しかし、普通にやるとエラーになる
eigen_val_vects_3 = eval_mat_3.eigenvects()

# 以下はエラー
# max(eigen_val_vects_3, key=(lambda x: x[EIGEN_VAL_IDX]))

In [26]:
# 固有値
eigen_val_vects_3[0][EIGEN_VAL_IDX]

In [27]:
# 固有値を実数部と虚数部に分ける
eigen_val_vects_3[0][EIGEN_VAL_IDX].as_real_imag()

In [28]:
# 実数部のみ取得
eigen_val_vects_3[0][EIGEN_VAL_IDX].as_real_imag()[0]

In [29]:
# 実数部のみを取得して、最大固有値とその時の多重度、固有ベクトルを取得
max(eigen_val_vects_3, key=(lambda x: x[EIGEN_VAL_IDX].as_real_imag()[0]))

⎛                    ⎡⎡2.46621207433047⎤⎤⎞
⎜                    ⎢⎢                ⎥⎥⎟
⎜3.8716922073687, 1, ⎢⎢2.02740066519113⎥⎥⎟
⎜                    ⎢⎢                ⎥⎥⎟
⎝                    ⎣⎣      1.0       ⎦⎦⎠

### 一貫性の評価

In [30]:
# 比較行列の、行数と列数を取得
eval_mat_3.shape

In [31]:
# 一貫性指標の計算
n = eval_mat_3.shape[0]
max_eigen_val = max(eigen_val_vects_3, key=(lambda x: x[EIGEN_VAL_IDX].as_real_imag()[EIGEN_VAL_IDX]))[EIGEN_VAL_IDX]
ci = (max_eigen_val - n) / (n - 1)
ci

In [32]:
# 西崎（2017）『意思決定の数理』を参考に、ランダム指標との比に基づいて評価する

# ランダム指標(初出はSaaty（2013）：
# 『Theory and Applications of the Analytic Network Process: Decision Making With Benefits, Opportunities, Costs, and Risks』)
ri = {3:0.52,
      4:0.89,
      5:1.11,
      6:1.25,
      7:1.35,
      8:1.40,
      9:1.45,
      10:1.49
     }

cr = ci / ri[n]
cr

In [33]:
# 重みベクトルを計算する関数を修正する
# 比較行列から重みベクトルを計算する関数
def calc_ahp_weight_vec_over3(comparison_mat):
    '''
    比較行列から重みベクトルを計算する関数
    一貫性の評価のための指標CIとCRも併せて算出して表示させる
    
    比較対象が3つ以上の時のみ使用できる

    Arguments:

    comparison_mat -- 比較行列
    '''
    
    # 固有値のインデックス
    EIGEN_VAL_IDX = 0
    
    # sympyの関数を使って、固有値と固有ベクトルを得る
    eigen_val_vects = comparison_mat.eigenvects()
    
    # 最大固有値とその時の固有ベクトルを取得
    # 固有値の大小は、実数部で評価
    eigen_val, multiplicity, eigen_vec = max(eigen_val_vects, key=(lambda x: x[EIGEN_VAL_IDX].as_real_imag()))
    
    # 計算過程を出力
    print("================================================================================")

    # ランダム指標(初出はSaaty（2013）：
    # 『Theory and Applications of the Analytic Network Process: Decision Making With Benefits, Opportunities, Costs, and Risks』)
    ri = {3:0.52,
          4:0.89,
          5:1.11,
          6:1.25,
          7:1.35,
          8:1.40,
          9:1.45,
          10:1.49
         }
    
    # 一貫性指標の計算
    n = comparison_mat.shape[0]
    ci = (eigen_val - n) / (n - 1)
    cr = ci / ri[n]

    print("一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください")
    print("比較対象数　　：", n)
    print("最大固有値　　：", N(eigen_val, 2))
    print("一貫性の指標CI：", N(ci, 2))
    print("一貫性の指標CR：", N(cr, 2))
    
    print("================================================================================")
    
    # 重みの合計が1になるように標準化
    weight_vec = eigen_vec[0] / sum(eigen_vec[0])
    
    # 重みを返す
    return(weight_vec)

In [34]:
# 基準の重みベクトル
eval_weight_3 = calc_ahp_weight_vec_over3(eval_mat_3)
N(eval_weight_3, 2)

一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください
比較対象数　　： 3
最大固有値　　： 3.9
一貫性の指標CI： 0.44
一貫性の指標CR： 0.84


⎡0.45⎤
⎢    ⎥
⎢0.37⎥
⎢    ⎥
⎣0.18⎦

### 寿司と牛丼、ラーメン問題を解決する

In [35]:
# 基準の行列
# 要素1が味
# 要素2が価格
# 要素3が時間
eval_mat_4 = Matrix(    
    [[1  , 3,    2],
     [1/3, 1,    1/2],
     [1/2, 2,    1]]
)

# 味に関する、お寿司と牛丼、ラーメンの比較行列
# 要素1がお寿司
# 要素2が牛丼
# 要素3がラーメン
taste_mat_4 = Matrix(    
    [[1  , 3,  2  ],
     [1/3, 1,  1/3],
     [1/2, 3,  1  ]]
)

# 価格に関する、お寿司と牛丼の比較行列
# 要素1がお寿司
# 要素2が牛丼
# 要素3がラーメン
price_mat_4 = Matrix(    
    [[1  , 1/9, 1/5],
     [9  , 1,   3  ],
     [5  , 1/3, 1  ]]
)

# 待ち時間に関する、お寿司と牛丼の比較行列
# 要素1がお寿司
# 要素2が牛丼
# 要素3がラーメン
time_mat_4 = Matrix(    
    [[1  , 1/9, 1/9],
     [9  , 1,   1  ],
     [9  , 1,   1  ]]
)

In [36]:
# 基準の重みベクトル
eval_weight_4 = calc_ahp_weight_vec_over3(eval_mat_4)
N(eval_weight_4, 2)

一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください
比較対象数　　： 3
最大固有値　　： 3.0
一貫性の指標CI： 0.0046
一貫性の指標CR： 0.0089


⎡0.54⎤
⎢    ⎥
⎢0.16⎥
⎢    ⎥
⎣0.3 ⎦

In [37]:
# 味に関する、お寿司と牛丼、ラーメンの重みベクトル
taste_weight_4 = calc_ahp_weight_vec_over3(taste_mat_4)
N(taste_weight_4, 2)

一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください
比較対象数　　： 3
最大固有値　　： 3.1
一貫性の指標CI： 0.027
一貫性の指標CR： 0.052


⎡0.53⎤
⎢    ⎥
⎢0.14⎥
⎢    ⎥
⎣0.33⎦

In [38]:
# 価格に関する、お寿司と牛丼、ラーメンの重みベクトル
price_weight_4 = calc_ahp_weight_vec_over3(price_mat_4)
N(price_weight_4, 2)

一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください
比較対象数　　： 3
最大固有値　　： 3.0
一貫性の指標CI： 0.015
一貫性の指標CR： 0.028


⎡0.063⎤
⎢     ⎥
⎢0.67 ⎥
⎢     ⎥
⎣0.27 ⎦

In [39]:
# 時間に関する、お寿司と牛丼、ラーメンの重みベクトル
time_weight_4 = calc_ahp_weight_vec_over3(time_mat_4)
N(time_weight_4, 2)

一貫性の評価。CI>0.1（あるいはCI>0.15）ならば、比較行列を修正してください
比較対象数　　： 3
最大固有値　　： 3.0
一貫性の指標CI： 0
一貫性の指標CR： 0


⎡0.053⎤
⎢     ⎥
⎢0.47 ⎥
⎢     ⎥
⎣0.47 ⎦

In [40]:
# 行が選択肢　；お寿司と牛丼、ラーメンの重みベクトル
# 列が基準：味・価格・時間
join_mat_4 = taste_weight_4.row_join(price_weight_4).row_join(time_weight_4)
N(join_mat_4, 2)

⎡0.53  0.063  0.053⎤
⎢                  ⎥
⎢0.14  0.67   0.47 ⎥
⎢                  ⎥
⎣0.33  0.27   0.47 ⎦

In [41]:
# 総合評価
# 要素1がお寿司
# 要素2が牛丼
# 要素3がラーメン
N(join_mat_4 * eval_weight_4, 2)

⎡0.31⎤
⎢    ⎥
⎢0.33⎥
⎢    ⎥
⎣0.36⎦